In [1]:
import ipywidgets as widgets
import numpy as np
import os
import ray
import subprocess
import tempfile
import time

from IPython.display import display

ray.init(redis_address=os.environ["REDIS_ADDRESS"])

{'local_scheduler_socket_names': ['/tmp/scheduler84824091'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store39630539', manager_name='/tmp/plasma_manager75845157', manager_port=64435)],
 'redis_address': '127.0.0.1:57463'}

#### Evaluate the box below to search for objects.

In [2]:
object_search = widgets.Text(
    value="",
    placeholder="Object ID",
    description="Search for an object:",
    disabled=False
)
display(object_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.object_table(object_search.value))

object_search.on_submit(handle_submit)

#### Evaluate the box below to search for tasks.

In [3]:
from IPython.display import display, IFrame
import shutil
 
def write_document(json_url, html_url):
    with open('index.html') as f:
        data = f.read()
    data = data.replace('REPLACE_THIS_STRING', json_url)
    with open(html_url, 'w+') as f:
        f.write(data)
        
task_search = widgets.Text(
    value="",
    placeholder="Task ID",
    description="Search for a task:",
    disabled=False
)
display(task_search)

task_table = ray.global_state.task_table() 

def handle_submit(sender):
    tmp = tempfile.mktemp() + ".json"
    tmp2 = tempfile.mktemp() + ".html"
    html_file = tmp2
    nodes_x, objects, edges = ray.global_state.computation_graph(task_search.value)
    path = os.path.expanduser("~/Desktop/test.json")
    ray.global_state.dump_catapult_trace(tmp, nodes_x, edges=edges, objects=objects)
    shutil.copy(tmp, 'tmp_trace.json')
    write_document('tmp_trace.json', 'to_display.html')
    display(IFrame('to_display.html', 900, 800))
      

task_search.on_submit(handle_submit)

In [4]:
# Instances of this class maintains keep track of whether or not a
# callback is currently executing. Since the execution of the callback
# may trigger more calls to the callback, this is used to prevent infinite
# recursions.
class _EventRecursionContextManager(object):
    def __init__(self):
        self.should_recurse = True
   
    def __enter__(self):
        self.should_recurse = False
       
    def __exit__(self, *args):
        self.should_recurse = True

total_time_value = "% total time"
total_tasks_value = "% total tasks"

# Function that returns instances of sliders and handles associated events.
def get_sliders(update):
    # Start_box value indicates the desired start point of queried window.
    start_box = widgets.FloatText(
        description="Start Time:",
        disabled=True,
    )
    
    # End_box value indicates the desired end point of queried window.
    end_box = widgets.FloatText(
        description="End Time:",
        disabled=True,
    )
    
    # Percentage slider. Indicates either % of total time or total tasks
    # depending on what breakdown_opt is set to.
    range_slider = widgets.IntRangeSlider(
        value=[70, 100],
        min=0,
        max=100,
        step=1,
        description="%:",
        continuous_update=False,
        orientation="horizontal",
        readout=True,
        readout_format=".0i%",
    )
    
    # Indicates the number of tasks that the user wants to be returned. Is
    # disabled when the breakdown_opt value is set to total_time_value. 
    num_tasks_box = widgets.IntText(
        description="Num Tasks:",
        disabled=False
    )
    
    # Dropdown bar that lets the user choose between modifying % of total
    # time or total number of tasks.
    breakdown_opt = widgets.Dropdown(
        options=[total_time_value, total_tasks_value],
        value=total_tasks_value,
        description="Selection Options:"
    )
   
    # Initially passed in to the update_wrapper function.
    INIT_EVENT = "INIT"
    
    # Create instance of context manager to determine whether callback is
    # currently executing 
    out_recursion = _EventRecursionContextManager()
   
    def update_wrapper(event):
        # Feature received a callback, but it shouldn't be executed
        # because the callback was the result of a different feature
        # executing its callback based on user input. 
        if not out_recursion.should_recurse:
            return
        
        # Feature received a callback and it should be executed because
        # the callback was the result of user input. 
        with out_recursion:
            smallest, largest, num_tasks = ray.global_state._job_length()
            diff = largest - smallest
            if num_tasks is not 0: 
                
                # Describes the initial values that the slider/text box
                # values should be set to.
                if event == INIT_EVENT:
                    if breakdown_opt.value == total_tasks_value:
                        num_tasks_box.value = -min(10000, num_tasks)
                        range_slider.value = (int(100 - (100. * -num_tasks_box.value) / num_tasks), 100)
                    else:
                        low, high = map(lambda x: x / 100., range_slider.value)
                        start_box.value = round(diff * low, 2)
                        end_box.value = round(diff * high, 2)
                        
                # Event was triggered by a change in the start_box value. 
                elif event["owner"] == start_box:
                    if start_box.value > end_box.value:
                        start_box.value = end_box.value
                    elif start_box.value < 0:
                        start_box.value = 0
                    low, high = range_slider.value
                    range_slider.value = (int((start_box.value * 100.) / diff), high)
                    
                # Event was triggered by a change in the end_box value. 
                elif event["owner"] == end_box:
                    if start_box.value > end_box.value:
                        end_box.value = start_box.value
                    elif end_box.value > diff:
                        end_box.value = diff
                    low, high = range_slider.value
                    range_slider.value = (low, int((end_box.value * 100.) / diff))
                    
                # Event was triggered by a change in the breakdown options
                # toggle.
                elif event["owner"] == breakdown_opt:
                    if breakdown_opt.value == total_tasks_value:
                        start_box.disabled = True
                        end_box.disabled = True
                        num_tasks_box.disabled = False
                        num_tasks_box.value = min(10000, num_tasks)
                        range_slider.value = (int(100 - (100. * num_tasks_box.value) / num_tasks), 100)
                    else:
                        start_box.disabled = False
                        end_box.disabled = False
                        num_tasks_box.disabled = True
                        range_slider.value = (int((start_box.value * 100.) / diff),
                                              int((end_box.value * 100.) / diff))
                        
                # Event was triggered by a change in the range_slider
                # value.
                elif event["owner"] == range_slider:
                    low, high = map(lambda x: x / 100., range_slider.value)
                    if breakdown_opt.value == total_tasks_value:
                        old_low, old_high = event["old"]
                        new_low, new_high = event["new"]
                        if old_low != new_low:
                            range_slider.value = (new_low, 100)
                            num_tasks_box.value = -(100. - new_low) / 100. * num_tasks
                        else:
                            range_slider.value = (0, new_high)
                            num_tasks_box.value = new_high / 100. * num_tasks
                    else:
                        start_box.value = round(diff * low, 2)
                        end_box.value = round(diff * high, 2)
                        
                # Event was triggered by a change in the num_tasks_box
                # value.
                elif event["owner"] == num_tasks_box:
                    if num_tasks_box.value > 0:
                        range_slider.value = (0, int(100 * float(num_tasks_box.value) / num_tasks))
                    elif num_tasks_box.value < 0:
                        range_slider.value = (100 + int(100 * float(num_tasks_box.value) / num_tasks), 100)
               
                if not update:
                    return

                diff = largest - smallest
                
                # Low and high are used to scale the times that are
                # queried to be relative to the absolute time. 
                low, high = map(lambda x: x / 100., range_slider.value)
                
                # Queries to task_profiles based on the slider and text
                # box values.
                # (Querying based on the % total amount of time.) 
                if breakdown_opt.value == total_time_value:
                    tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
                    
                # (Querying based on % of total number of tasks that were
                # run.)
                elif breakdown_opt.value == total_tasks_value:
                    if range_slider.value[0] == 0:
                        tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * high), fwd=True)
                    else:
                        tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * (high - low)), fwd=False)

                update(smallest, largest, num_tasks, tasks)
    
    # Get updated values from a slider or text box, and update the rest of
    # them accordingly.
    range_slider.observe(update_wrapper, names="value")
    breakdown_opt.observe(update_wrapper, names="value")
    start_box.observe(update_wrapper, names="value")
    end_box.observe(update_wrapper, names="value")
    num_tasks_box.observe(update_wrapper, names="value")
    
    # Initializes the sliders
    update_wrapper(INIT_EVENT)
    
    # Display sliders and search boxes 
    display(start_box, end_box, range_slider, num_tasks_box, breakdown_opt)
   
    # Return the sliders and text boxes 
    return start_box, end_box, range_slider, breakdown_opt

In [5]:
def task_timeline():
    path_input = widgets.Button(description="View task timeline")

    breakdown_basic = "Basic"
    breakdown_task = "Task Breakdowns"
    
    breakdown_opt = widgets.Dropdown(
        options=["Basic", "Task Breakdowns"],
        value="Basic",
        description="View options:",
        disabled=False,
    )
    
    start_box, end_box, range_slider, time_opt = get_sliders(False)
    display(breakdown_opt)
    display(path_input)

    def find_trace2html():
        trace2html = "/tmp/ray/catapult/tracing/bin/trace2html"
        # Clone the catapult repository if it doesn't exist. TODO(rkn): We
        # could do this in the build.sh script later on.
        if not os.path.exists(trace2html): 
            cmd = ["git", "clone", "https://github.com/catapult-project/catapult.git", "/tmp/ray/catapult"]
            subprocess.check_output(cmd) 
            print("Cloning catapult to /tmp/ray/catapult.")
        assert os.path.exists(trace2html)
        return trace2html

    def handle_submit(sender):
        tmp = tempfile.mktemp() + ".json"
        tmp2 = tempfile.mktemp() + ".html"
        
        if breakdown_opt.value == breakdown_basic:
            breakdown = False
        elif breakdown_opt.value == breakdown_task:
            breakdown = True
        else:
            raise ValueError("Unexpected breakdown value '{}'".format(breakdown_opt.value))
            
        low, high = map(lambda x: x / 100., range_slider.value)
        
        smallest, largest, num_tasks = ray.global_state._job_length()
        diff = largest - smallest

        if time_opt.value == total_time_value:
            tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
        elif time_opt.value == total_tasks_value:
            if range_slider.value[0] == 0:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * high), fwd=True)
            else:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * (high - low)), fwd=False)
        else:
            raise ValueError("Unexpected time value '{}'".format(time_opt.value))
        
        print("{} tasks to trace".format(len(tasks)))
        print("Dumping task profiling data to " + tmp)
        ray.global_state.dump_catapult_trace(tmp, tasks, breakdowns=breakdown)
        print("Converting chrome trace to " + tmp2)
        trace2html = find_trace2html()
        # TODO(rkn): The trace2html script currently requires Python 2.
        # Remove this dependency.
        subprocess.check_output(["python2", trace2html, tmp, "--output", tmp2])
        # Open the timeline in Chrome. TODO(rkn): We should remove the
        # dependency on Chrome and use whatever browser is currently being
        # used. Note that this currently does not work when Ray is being
        # used on a cluster and the browser is running locally.
        print("Opening html file in browser...")
        subprocess.Popen(["open", "-a", "Google Chrome", tmp2])

    path_input.on_click(handle_submit)

task_timeline()

#### Task duration distribution plot.

In [6]:
from bokeh.models import Range1d, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, helpers
from bokeh.io import output_notebook, push_notebook
from bokeh.resources import CDN
output_notebook(resources=CDN)


def task_completion_time_distribution():
    # Create the Bokeh plot 
    p = figure(title="Task Completion Time Distribution",tools=["save", "hover", "wheel_zoom", "box_zoom", "pan"],
                background_fill_color="#FFFFFF", x_range=(0, 1), y_range = (0, 1))
    
    # Create the data source that the plot pulls from 
    source = ColumnDataSource(data={
        "top": [],
        "left": [],
        "right": []
    })
    
    # Plot the histogram rectangles
    p.quad(top="top", bottom=0, left="left", right="right", source=source,
            fill_color="#B3B3B3", line_color="#033649")

    # Label the plot axes 
    p.xaxis.axis_label = "Duration in seconds"
    p.yaxis.axis_label = "Number of tasks"

    handle = show(gridplot(p, ncols=1, plot_width=500, plot_height=500, toolbar_location="below"),
                 notebook_handle=True)

    # Function to update the plot 
    def task_completion_time_update(abs_earliest, abs_latest, abs_num_tasks, tasks):
        if len(tasks) == 0:
            return
        
        # Create the distribution to plot
        distr = []
        for task_id, data in tasks.items():
            distr.append(data["store_outputs_end"] - data["get_arguments_start"])
            
        # Create a histogram from the distribution 
        top, bin_edges = np.histogram(distr, bins="auto")
        left = bin_edges[:-1]
        right = bin_edges[1:]

        source.data = {"top": top, "left": left, "right": right}
        
        # Set the x and y ranges 
        x_range = (min(left) if len(left) else 0, max(right) if len(right) else 1)
        y_range = (0, max(top) + 1 if len(top) else 1)
        
        x_range = helpers._get_range(x_range)
        p.x_range.start = x_range.start
        p.x_range.end = x_range.end

        y_range = helpers._get_range(y_range)
        p.y_range.start = y_range.start
        p.y_range.end = y_range.end
        
        # Push updates to the plot
        push_notebook(handle=handle)
    
    get_sliders(task_completion_time_update)

task_completion_time_distribution()

Loading BokehJS ...

In [8]:
from IPython.display import display, IFrame
import shutil
 
def write_document(json_url, html_url):
    with open('index.html') as f:
        data = f.read()
    data = data.replace('REPLACE_THIS_STRING', json_url)
    with open(html_url, 'w+') as f:
        f.write(data)

def task_timeline():
    path_input = widgets.Button(description="View task timeline")
 
    breakdown_basic = "Basic"
    breakdown_task = "Task Breakdowns"
   
    breakdown_opt = widgets.Dropdown(
        options=["Basic", "Task Breakdowns"],
        value="Basic",
        description="View options:",
        disabled=False,
    )
   
    start_box, end_box, range_slider, time_opt = get_sliders(False)
    display(breakdown_opt)
    display(path_input)
   
    def find_trace2html():
        trace2html = "/tmp/ray/catapult/tracing/bin/trace2html"
        # Clone the catapult repository if it doesn't exist. TODO(rkn): We
        # could do this in the build.sh script later on.
        if not os.path.exists(trace2html):
            cmd = ["git", "clone", "https://github.com/catapult-project/catapult.git", "/tmp/ray/catapult"]
            subprocess.check_output(cmd)
            print("Cloning catapult to /tmp/ray/catapult.")
        assert os.path.exists(trace2html)
        return trace2html
 
    def handle_submit(sender):
        tmp = tempfile.mktemp() + ".json"
        tmp2 = tempfile.mktemp() + ".html"
        html_file = tmp2
        if breakdown_opt.value == breakdown_basic:
            breakdown = False
        elif breakdown_opt.value == breakdown_task:
            breakdown = True
        else:
            raise ValueError("Unexpected breakdown value '{}'".format(breakdown_opt.value))
           
        low, high = map(lambda x: x / 100., range_slider.value)
       
        smallest, largest, num_tasks = ray.global_state._job_length()
        diff = largest - smallest
 
        if time_opt.value == total_time_value:
            tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
        elif time_opt.value == total_tasks_value:
            if range_slider.value[0] == 0:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * high), fwd=True)
            else:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * (high - low)), fwd=False)
        else:
            raise ValueError("Unexpected time value '{}'".format(time_opt.value))
       
        print("{} tasks to trace".format(len(tasks)))
        nodes, objects = ray.global_state.computation_graph()
        ray.global_state.dump_catapult_trace(tmp, tasks, objects=objects)
        shutil.copy(tmp, 'tmp_trace.json')
        write_document('tmp_trace.json', 'to_display.html')
        display(IFrame('to_display.html', 900, 800))
      
 
    path_input.on_click(handle_submit)

task_timeline()

20 tasks to trace
Creating JSON 80/20
